In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import sys
import os
import re
import random

from time import time
from tqdm import tqdm

from contextlib import contextmanager
import lightgbm as lgb

In [ ]:
#上限表示数を拡張
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 1000)

In [ ]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

In [ ]:
df_6_tr = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_train_rinna_clip.csv")
df_6_ts = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_test_rinna_clip.csv")

In [ ]:
df_6_tr = df_6_tr.drop(["Unnamed: 0","odai_photo_file_name","is_laugh","id","text","img_path"], axis=1)

In [ ]:
df_6_ts = df_6_ts.drop(["Unnamed: 0","odai_photo_file_name","id","text","img_path"], axis=1)

In [ ]:
tr_text_embed = df_6_tr.loc[:,"clip-text0":"clip-text511"]

In [ ]:
tr_text_embed_np = tr_text_embed.to_numpy()

In [ ]:
tr_image_embed = df_6_tr.loc[:,"clip_image_0":"clip_image_511"]

In [ ]:
tr_image_embed_np = tr_image_embed.to_numpy()

In [ ]:
ts_text_embed = df_6_ts.loc[:,"clip-text0":"clip-text511"]

In [ ]:
ts_text_embed_np = ts_text_embed.to_numpy()

In [ ]:
ts_image_embed = df_6_ts.loc[:,"clip_image_0":"clip_image_511"]

In [ ]:
ts_image_embed_np = ts_image_embed.to_numpy()

In [ ]:
del df_6_tr,df_6_ts,tr_image_embed,tr_text_embed,ts_image_embed,ts_text_embed

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity 

cos_sim_tr = cosine_similarity(tr_image_embed_np , tr_text_embed_np)

In [ ]:
cos_sim_ts = cosine_similarity(ts_image_embed_np , ts_text_embed_np)

In [ ]:
cos_sim_tr = sc.fit_transform(cos_sim_tr)

In [ ]:
cos_sim_ts = sc.fit_transform(cos_sim_ts)

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state = 0)
features_tr = tsne.fit_transform(cos_sim_tr)

In [ ]:
del cos_sim_tr

In [ ]:
features_ts = tsne.fit_transform(cos_sim_ts)

In [ ]:
del cos_sim_ts

In [ ]:
print(features_tr)

In [ ]:
#x = np.argsort(-cos_sim_tr, axis=0)

In [ ]:
#x2 = np.argsort(-cos_sim_ts, axis=0)

In [ ]:
#x_tr_top3 = x[0:3].mean()
#x_tr_top10 = x[0:10].mean()

In [ ]:
#x_ts_top3 = x2[0:3].mean()
#x_ts_top10 = x2[0:3].mean()

In [ ]:
#print(x[0:3])

In [ ]:
cos_sim_tr_df = pd.DataFrame(features_tr).add_prefix("cos_sim_tr")
cos_sim_ts_df = pd.DataFrame(features_ts).add_prefix("cos_sim_ts")

In [ ]:
cos_sim_tr_df = cos_sim_tr_df.to_csv('/content/drive/MyDrive/nishika/output/tsne_cos_sim_tr_df.csv')

In [ ]:
cos_sim_ts_df = cos_sim_ts_df.to_csv('/content/drive/MyDrive/nishika/output/tsne_cos_sim_ts_df.csv')